In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import CategoricalEncoder
import category_encoders as ce
import collections as col
from cvxpy import *

In [2]:
air_store = pd.read_csv('data/air_store_info.csv')

date_info = pd.read_csv('data/date_info.csv')
date_info_drop = date_info.drop('day_of_week', axis=1)
date_info_drop.columns.values[0]='visit_date'

store_id = pd.read_csv('data/store_id_relation.csv', index_col='air_store_id')
air_visit1 = pd.read_csv('data/air_visit_data.csv')
air_reserve1 = pd.read_csv('data/air_reserve.csv')

weather_data1 = pd.read_csv('data/merged_rest_data_weather_measurements.csv')
weather_data = weather_data1[['air_store_id', 
                              'visit_date', 
                              'avg_temperature1', 
                              'high_temperature1', 
                              'low_temperature1',
                              'hours_sunlight1']]

In [3]:
# drop duplicates in air_reserve data, and sum according to dates
air_reserve = air_reserve1[air_reserve1.duplicated()==False]
#air_reserve.duplicated().sum()
air_reserve['visit_datetime'] = air_reserve['visit_datetime'].apply(lambda _: _[:10])
air_reserve = air_reserve.drop(['reserve_datetime'], axis=1)\
              .groupby(['air_store_id', 'visit_datetime'])['reserve_visitors']\
              .sum().reset_index().rename(columns = {'visit_datetime':'visit_date'})

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [4]:
id_pool = list(set(air_visit1['air_store_id']))
dictionary = {}
for storeid in id_pool:
    dictionary[storeid] = air_visit1.loc[air_visit1['air_store_id'] == storeid]
    dictionary[storeid]['minus7days'] = dictionary[storeid]['visitors'].shift(7)
    dictionary[storeid]['minus1days'] = dictionary[storeid]['visitors'].shift(1)
    dictionary[storeid]['MA'] = dictionary[storeid].rolling(window=7)['minus1days'].mean()
frames = []
for storeid in id_pool:
    frames.append(dictionary[storeid])
df = pd.concat(frames).dropna(axis=0, how='any')

tmp0 = pd.merge(df,
                date_info_drop,
                on='visit_date',
                how = 'left')
tmp0['holiday_flg'] = tmp0['holiday_flg'].astype('category')
tmp1 = pd.merge(tmp0,
                air_reserve,
                on=['air_store_id', 'visit_date'],
                how = 'left')
tmp2 = pd.merge(tmp1,
                air_store,
                on='air_store_id',
                how = 'left')
tmp3 = pd.merge(tmp2,
               weather_data,
               on=['air_store_id', 'visit_date'],
               how = 'left')
air_visit_reserve_merge = tmp3.dropna(axis=0, how='any').drop_duplicates()

air_visit_reserve_merge['visit_date'] = pd.to_datetime(air_visit_reserve_merge['visit_date'])
#air_visit_reserve_merge['month'] = air_visit_reserve_merge['visit_date'].dt.month
air_visit_reserve_merge['day'] = air_visit_reserve_merge['visit_date'].dt.day
air_visit_reserve_merge['weekday'] = air_visit_reserve_merge['visit_date'].dt.weekday

#air_visit_reserve_merge['month'] = air_visit_reserve_merge['month'].astype('category')
air_visit_reserve_merge['day'] = air_visit_reserve_merge['day'].astype('category')
air_visit_reserve_merge['weekday'] = air_visit_reserve_merge['weekday'].astype('category')

onehot_day_weekday = pd.get_dummies(air_visit_reserve_merge[['day', 'weekday']])

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

In [85]:
import seaborn as sns
# names = ['minus7days', 
#          'minus1days', 
#          'MA', 
#          'reserve_visitors', 
#          'avg_temperature1', 
#          'high_temperature1', 
#          'low_temperature1', 
#          'hours_sunlight1']
sns.heatmap(air_visit_reserve_merge[['minus7days', 
         'minus1days', 
         'MA', 
         'reserve_visitors', 
         'avg_temperature1', 
         'high_temperature1', 
         'low_temperature1', 
         'hours_sunlight1']])

# for name in names:
#     print(name, air_visit_reserve_merge['visitors'].corr(air_visit_reserve_merge[name]))

KeyboardInterrupt: 

In [74]:
geodate = pd.pivot_table(air_visit_reserve_merge[['visit_date', 'visitors', 'air_area_name']], 
               index='visit_date', 
               columns=['air_area_name'], 
               aggfunc=np.sum).fillna(0)

In [75]:
import plotly
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username="stophy", api_key="NHvy5Q2HvguCvey9KwXd")

data = [go.Surface(z=geodate.values.tolist(), colorscale='Viridis')]

layout = go.Layout(
    width=800,
    height=700,
    autosize=False,
    title='geo_weekday',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        aspectratio = dict( x=1, y=1, z=0.7 ),
        aspectmode = 'manual'
    )
)

fig = dict(data=data, layout=layout)

plotly.plotly.iplot(fig, filename='geodate')

In [5]:
geoweekday = pd.pivot_table(air_visit_reserve_merge[['weekday', 'visitors', 'air_area_name']], 
               index='weekday', 
               columns=['air_area_name'], 
               aggfunc=np.sum).fillna(0)

area_list=[]
for i in [0,6,9,12,20,37]:
    area_list.append(geoweekday.columns.tolist()[i][1])

l1 = air_visit_reserve_merge['air_area_name'].tolist()
l2 = air_visit_reserve_merge['weekday'].tolist()
l1bool = [elem in area_list for elem in l1]
l2bool = [elem==5 for elem in l2]
lbool = [int(l1bool[i]&l2bool[i]) for i in range(len(l1))]
air_visit_reserve_merge['geotemp'] = lbool

In [34]:
import plotly
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username="stophy", api_key="NHvy5Q2HvguCvey9KwXd")

data = [go.Surface(z=geoweekday.values.tolist(), colorscale='Viridis')]

layout = go.Layout(
    width=800,
    height=700,
    autosize=False,
    title='geo_weekday',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        aspectratio = dict( x=1, y=1, z=0.7 ),
        aspectmode = 'manual'
    )
)

fig = dict(data=data, layout=layout)

plotly.plotly.iplot(fig, filename='geoweekday')

In [32]:
geoday = pd.pivot_table(air_visit_reserve_merge[['day', 'visitors', 'air_area_name']], 
               index='day', 
               columns=['air_area_name'], 
               aggfunc=np.sum).fillna(0)

In [33]:
import plotly
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username="stophy", api_key="NHvy5Q2HvguCvey9KwXd")

data = [go.Surface(z=geoday.values.tolist(), colorscale='Viridis')]

layout = go.Layout(
    width=800,
    height=700,
    autosize=False,
    title='geo_day',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        aspectratio = dict( x=1, y=1, z=0.7 ),
        aspectmode = 'manual'
    )
)

fig = dict(data=data, layout=layout)

plotly.plotly.iplot(fig, filename='geoday')

In [5]:
air_visit_reserve_merge['air_genre_name'] =\
    air_visit_reserve_merge['air_genre_name'].astype('category')
air_visit_reserve_merge['air_area_name'] =\
    air_visit_reserve_merge['air_area_name'].astype('category')

In [6]:
le_genre = LabelEncoder()
le_genre.fit(air_visit_reserve_merge['air_genre_name'])
df_genre_label_encoding = pd.DataFrame(le_genre.transform(air_visit_reserve_merge['air_genre_name']),
                                       columns=['genre_label_encoding'])

le_area = LabelEncoder()
le_area.fit(air_visit_reserve_merge['air_area_name'])
df_area_label_encoding = pd.DataFrame(le_area.transform(air_visit_reserve_merge['air_area_name']),
                                       columns=['area_label_encoding'])

df_label_encoding = pd.merge(df_genre_label_encoding, df_area_label_encoding, left_index=True, right_index=True)
df_label_encoding['genre_label_encoding'] = df_label_encoding['genre_label_encoding'].astype('category')
df_label_encoding['area_label_encoding'] = df_label_encoding['area_label_encoding'].astype('category')

In [7]:
df_label_encoding.index = air_visit_reserve_merge.index

df_with_label_encoding = pd.merge(air_visit_reserve_merge, 
                                  df_label_encoding, 
                                  right_index=True, left_index=True)\
                            .dropna(axis=0, how='any')\
                            .drop(['air_genre_name',
                                    'air_area_name',
                                    'latitude',
                                    'longitude'], axis=1)

In [8]:
df_with_onehot1 = pd.get_dummies(air_visit_reserve_merge, 
                               columns=['air_genre_name','air_area_name'])\
                    .dropna(axis=0, how='any')\
                    .drop(['latitude',
                            'longitude'], axis=1)

In [9]:
df_with_onehot = pd.merge(df_with_onehot1, 
         onehot_day_weekday, 
         how='outer', 
         left_index=True, 
         right_index=True).drop(['day','weekday'], 
                                axis=1)

In [10]:
encoder_target = ce.TargetEncoder(cols = ['air_genre_name', 'air_area_name'])
tmp_encoder_target = encoder_target.fit_transform(air_visit_reserve_merge[['air_genre_name', 
                                                                           'air_area_name']],
                                                  air_visit_reserve_merge['visitors'])
tmp_encoder_target = tmp_encoder_target.rename(index=str, 
                                               columns={'air_genre_name':'air_genre_name_encoding',
                                                        'air_area_name':'air_area_name_encoding'})
tmp_encoder_target['air_genre_name_encoding'] = tmp_encoder_target['air_genre_name_encoding'].astype('category')
tmp_encoder_target['air_area_name_encoding'] = tmp_encoder_target['air_area_name_encoding'].astype('category')

In [11]:
tmp_encoder_target.index = tmp_encoder_target.index.map(int)
df_with_target_encoding = pd.concat([air_visit_reserve_merge, tmp_encoder_target], axis=1)\
                            .dropna(axis=0, how='any')\
                            .drop(['air_genre_name',
                                    'air_area_name',
                                    'latitude',
                                    'longitude'], axis=1)

In [12]:
df_baseline = air_visit_reserve_merge[['air_store_id', 'visit_date', 'visitors', 'minus7days']]
df_baseline['intercept'] = 1

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
def designed_train_test_split(df):
    df['visit_date'] = df['visit_date'].astype(str)
    test_date_selector = df["visit_date"].str.startswith("2017-04")
    val_date_selector = df["visit_date"].str.startswith("2017-03")
    df_test = df[test_date_selector].drop(['air_store_id', 'visit_date'], axis=1)
    df_val = df[val_date_selector].drop(['air_store_id', 'visit_date'], axis=1)
    df_train = df[~(test_date_selector | val_date_selector)].drop(['air_store_id', 'visit_date'], axis=1)
    
    df_train_y = df_train['visitors']
    df_train_X = df_train.drop(['visitors'], axis=1)
    df_val_y = df_val['visitors']
    df_val_X = df_val.drop(['visitors'], axis=1)
    df_test_y = df_test['visitors'] #true y
    df_test_X = df_test.drop(['visitors'], axis=1)     

    return df_train_X, df_train_y, df_val_X, df_val_y, df_test_X, df_test_y

In [16]:
TrainValTest_dict = col.OrderedDict([
    ("df_train_X", []),
    ("df_train_y", []),
    ("df_val_X", []),
    ("df_val_y", []),
    ("df_test_X", []),
    ("df_test_y", []),
])
df_name_list = [df_baseline, df_with_target_encoding, df_with_onehot, df_with_label_encoding]
for name in df_name_list:
    tmp_train_X, tmp_train_y, tmp_val_X, tmp_val_y, tmp_test_X, tmp_test_y = designed_train_test_split(name)
    TrainValTest_dict['df_train_X'].append(tmp_train_X)
    TrainValTest_dict['df_train_y'].append(tmp_train_y)
    TrainValTest_dict['df_val_X'].append(tmp_val_X)
    TrainValTest_dict['df_val_y'].append(tmp_val_y)
    TrainValTest_dict['df_test_X'].append(tmp_test_X)
    TrainValTest_dict['df_test_y'].append(tmp_test_y)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
def score_func(prediction, true):
    num_samples = true.shape[0]
    evaluation = np.linalg.norm(np.divide(prediction-true, true), ord=1)/num_samples
    return evaluation

In [22]:
import lightgbm as lgb

lgb_train = lgb.Dataset(TrainValTest_dict['df_train_X'][3], TrainValTest_dict['df_train_y'][3])
lgb_eval = lgb.Dataset(TrainValTest_dict['df_val_X'][3], TrainValTest_dict['df_val_y'][3], reference=lgb_train)

scores = []
for i in np.linspace(0.01, 1.5, 15):
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'mape',
        #'num_leaves': 50,
        'learning_rate': i,
        #'max_depth': 10
    }

    gbm = lgb.train(params, 
                   lgb_train, 
                   num_boost_round=500, 
                   valid_sets=lgb_eval, 
                   feature_name='auto', 
                   categorical_feature='auto', 
                   early_stopping_rounds=None,
                   )

    y_pred = gbm.predict(TrainValTest_dict['df_test_X'][3], num_iteration=gbm.best_iteration)
scores.append(score_func(y_pred, TrainValTest_dict['df_test_y'][3]))


/anaconda/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


[1]	valid_0's mape: 0.642097
[2]	valid_0's mape: 0.639122
[3]	valid_0's mape: 0.636174
[4]	valid_0's mape: 0.633243
[5]	valid_0's mape: 0.630347
[6]	valid_0's mape: 0.627473
[7]	valid_0's mape: 0.624603
[8]	valid_0's mape: 0.62179
[9]	valid_0's mape: 0.618979
[10]	valid_0's mape: 0.616241
[11]	valid_0's mape: 0.613482
[12]	valid_0's mape: 0.610752
[13]	valid_0's mape: 0.608055
[14]	valid_0's mape: 0.605385
[15]	valid_0's mape: 0.602751
[16]	valid_0's mape: 0.600188
[17]	valid_0's mape: 0.597608
[18]	valid_0's mape: 0.595045
[19]	valid_0's mape: 0.592506
[20]	valid_0's mape: 0.590159
[21]	valid_0's mape: 0.587695
[22]	valid_0's mape: 0.585256
[23]	valid_0's mape: 0.582836
[24]	valid_0's mape: 0.58044
[25]	valid_0's mape: 0.578276
[26]	valid_0's mape: 0.575992
[27]	valid_0's mape: 0.573713
[28]	valid_0's mape: 0.571452
[29]	valid_0's mape: 0.569294
[30]	valid_0's mape: 0.567062
[31]	valid_0's mape: 0.56485
[32]	valid_0's mape: 0.562864
[33]	valid_0's mape: 0.560709
[34]	valid_0's mape: 0

[276]	valid_0's mape: 0.393259
[277]	valid_0's mape: 0.393043
[278]	valid_0's mape: 0.392988
[279]	valid_0's mape: 0.392781
[280]	valid_0's mape: 0.39254
[281]	valid_0's mape: 0.392324
[282]	valid_0's mape: 0.392109
[283]	valid_0's mape: 0.391985
[284]	valid_0's mape: 0.391779
[285]	valid_0's mape: 0.391596
[286]	valid_0's mape: 0.391416
[287]	valid_0's mape: 0.391267
[288]	valid_0's mape: 0.3911
[289]	valid_0's mape: 0.390994
[290]	valid_0's mape: 0.390869
[291]	valid_0's mape: 0.390733
[292]	valid_0's mape: 0.390563
[293]	valid_0's mape: 0.390445
[294]	valid_0's mape: 0.390332
[295]	valid_0's mape: 0.390261
[296]	valid_0's mape: 0.390161
[297]	valid_0's mape: 0.389997
[298]	valid_0's mape: 0.389857
[299]	valid_0's mape: 0.389726
[300]	valid_0's mape: 0.389625
[301]	valid_0's mape: 0.389413
[302]	valid_0's mape: 0.389266
[303]	valid_0's mape: 0.389163
[304]	valid_0's mape: 0.388965
[305]	valid_0's mape: 0.388864
[306]	valid_0's mape: 0.388667
[307]	valid_0's mape: 0.388505
[308]	valid

[49]	valid_0's mape: 0.400316
[50]	valid_0's mape: 0.400317
[51]	valid_0's mape: 0.400365
[52]	valid_0's mape: 0.400323
[53]	valid_0's mape: 0.399845
[54]	valid_0's mape: 0.39922
[55]	valid_0's mape: 0.399211
[56]	valid_0's mape: 0.399107
[57]	valid_0's mape: 0.399126
[58]	valid_0's mape: 0.399015
[59]	valid_0's mape: 0.399036
[60]	valid_0's mape: 0.398935
[61]	valid_0's mape: 0.398922
[62]	valid_0's mape: 0.398963
[63]	valid_0's mape: 0.398907
[64]	valid_0's mape: 0.398936
[65]	valid_0's mape: 0.398968
[66]	valid_0's mape: 0.398891
[67]	valid_0's mape: 0.398875
[68]	valid_0's mape: 0.398852
[69]	valid_0's mape: 0.398869
[70]	valid_0's mape: 0.398811
[71]	valid_0's mape: 0.398821
[72]	valid_0's mape: 0.398835
[73]	valid_0's mape: 0.398854
[74]	valid_0's mape: 0.398843
[75]	valid_0's mape: 0.398828
[76]	valid_0's mape: 0.398835
[77]	valid_0's mape: 0.398905
[78]	valid_0's mape: 0.398924
[79]	valid_0's mape: 0.398924
[80]	valid_0's mape: 0.398877
[81]	valid_0's mape: 0.398857
[82]	valid_

[333]	valid_0's mape: 0.397197
[334]	valid_0's mape: 0.397189
[335]	valid_0's mape: 0.39718
[336]	valid_0's mape: 0.397179
[337]	valid_0's mape: 0.397173
[338]	valid_0's mape: 0.397177
[339]	valid_0's mape: 0.39717
[340]	valid_0's mape: 0.397166
[341]	valid_0's mape: 0.397159
[342]	valid_0's mape: 0.397146
[343]	valid_0's mape: 0.397115
[344]	valid_0's mape: 0.3971
[345]	valid_0's mape: 0.397097
[346]	valid_0's mape: 0.397103
[347]	valid_0's mape: 0.397149
[348]	valid_0's mape: 0.397143
[349]	valid_0's mape: 0.397153
[350]	valid_0's mape: 0.397155
[351]	valid_0's mape: 0.397155
[352]	valid_0's mape: 0.397141
[353]	valid_0's mape: 0.396695
[354]	valid_0's mape: 0.396711
[355]	valid_0's mape: 0.396701
[356]	valid_0's mape: 0.396691
[357]	valid_0's mape: 0.396709
[358]	valid_0's mape: 0.396716
[359]	valid_0's mape: 0.396748
[360]	valid_0's mape: 0.396763
[361]	valid_0's mape: 0.396792
[362]	valid_0's mape: 0.396786
[363]	valid_0's mape: 0.396788
[364]	valid_0's mape: 0.39678
[365]	valid_0

...

[356]	valid_0's mape: 0.482166
[357]	valid_0's mape: 0.482266
[358]	valid_0's mape: 0.481836
[359]	valid_0's mape: 0.484765
[360]	valid_0's mape: 0.484311
[361]	valid_0's mape: 0.484353
[362]	valid_0's mape: 0.483725
[363]	valid_0's mape: 0.483545
[364]	valid_0's mape: 0.487305
[365]	valid_0's mape: 0.487758
[366]	valid_0's mape: 0.487694
[367]	valid_0's mape: 0.487446
[368]	valid_0's mape: 0.4872
[369]	valid_0's mape: 0.487707
[370]	valid_0's mape: 0.486881
[371]	valid_0's mape: 0.487022
[372]	valid_0's mape: 0.485142
[373]	valid_0's mape: 0.49982
[374]	valid_0's mape: 0.49943
[375]	valid_0's mape: 0.501572
[376]	valid_0's mape: 0.503457
[377]	valid_0's mape: 0.502989
[378]	valid_0's mape: 0.504674
[379]	valid_0's mape: 0.508209
[380]	valid_0's mape: 0.516815
[381]	valid_0's mape: 0.515814
[382]	valid_0's mape: 0.525761
[383]	valid_0's mape: 0.530643
[384]	valid_0's mape: 0.530675
[385]	valid_0's mape: 0.530232
[386]	valid_0's mape: 0.545244
[387]	valid_0's mape: 0.541455
[388]	valid_

[128]	valid_0's mape: 0.47066
[129]	valid_0's mape: 0.470625
[130]	valid_0's mape: 0.470827
[131]	valid_0's mape: 0.470916
[132]	valid_0's mape: 0.470717
[133]	valid_0's mape: 0.472078
[134]	valid_0's mape: 0.472283
[135]	valid_0's mape: 0.472371
[136]	valid_0's mape: 0.47294
[137]	valid_0's mape: 0.471889
[138]	valid_0's mape: 0.47133
[139]	valid_0's mape: 0.47132
[140]	valid_0's mape: 0.471726
[141]	valid_0's mape: 0.471694
[142]	valid_0's mape: 0.471411
[143]	valid_0's mape: 0.470952
[144]	valid_0's mape: 0.471619
[145]	valid_0's mape: 0.472256
[146]	valid_0's mape: 0.472241
[147]	valid_0's mape: 0.47158
[148]	valid_0's mape: 0.472
[149]	valid_0's mape: 0.471542
[150]	valid_0's mape: 0.471998
[151]	valid_0's mape: 0.471586
[152]	valid_0's mape: 0.471325
[153]	valid_0's mape: 0.470858
[154]	valid_0's mape: 0.47106
[155]	valid_0's mape: 0.471241
[156]	valid_0's mape: 0.471889
[157]	valid_0's mape: 0.472308
[158]	valid_0's mape: 0.472412
[159]	valid_0's mape: 0.473256
[160]	valid_0's m

[395]	valid_0's mape: 0.496053
[396]	valid_0's mape: 0.496007
[397]	valid_0's mape: 0.495696
[398]	valid_0's mape: 0.495348
[399]	valid_0's mape: 0.495153
[400]	valid_0's mape: 0.495007
[401]	valid_0's mape: 0.494995
[402]	valid_0's mape: 0.495265
[403]	valid_0's mape: 0.495682
[404]	valid_0's mape: 0.495566
[405]	valid_0's mape: 0.49591
[406]	valid_0's mape: 0.495879
[407]	valid_0's mape: 0.496006
[408]	valid_0's mape: 0.496026
[409]	valid_0's mape: 0.495973
[410]	valid_0's mape: 0.496354
[411]	valid_0's mape: 0.496236
[412]	valid_0's mape: 0.496235
[413]	valid_0's mape: 0.495912
[414]	valid_0's mape: 0.495942
[415]	valid_0's mape: 0.49675
[416]	valid_0's mape: 0.496672
[417]	valid_0's mape: 0.496651
[418]	valid_0's mape: 0.496923
[419]	valid_0's mape: 0.497032
[420]	valid_0's mape: 0.497141
[421]	valid_0's mape: 0.49692
[422]	valid_0's mape: 0.49695
[423]	valid_0's mape: 0.496807
[424]	valid_0's mape: 0.496749
[425]	valid_0's mape: 0.49692
[426]	valid_0's mape: 0.496667
[427]	valid_0

[174]	valid_0's mape: 0.483498
[175]	valid_0's mape: 0.482896
[176]	valid_0's mape: 0.482295
[177]	valid_0's mape: 0.482206
[178]	valid_0's mape: 0.482187
[179]	valid_0's mape: 0.482394
[180]	valid_0's mape: 0.482474
[181]	valid_0's mape: 0.482807
[182]	valid_0's mape: 0.483256
[183]	valid_0's mape: 0.482976
[184]	valid_0's mape: 0.4826
[185]	valid_0's mape: 0.483153
[186]	valid_0's mape: 0.482981
[187]	valid_0's mape: 0.48331
[188]	valid_0's mape: 0.483131
[189]	valid_0's mape: 0.483461
[190]	valid_0's mape: 0.48384
[191]	valid_0's mape: 0.482544
[192]	valid_0's mape: 0.483282
[193]	valid_0's mape: 0.483382
[194]	valid_0's mape: 0.4831
[195]	valid_0's mape: 0.483669
[196]	valid_0's mape: 0.483635
[197]	valid_0's mape: 0.482979
[198]	valid_0's mape: 0.483397
[199]	valid_0's mape: 0.483254
[200]	valid_0's mape: 0.483253
[201]	valid_0's mape: 0.483623
[202]	valid_0's mape: 0.483471
[203]	valid_0's mape: 0.484587
[204]	valid_0's mape: 0.484615
[205]	valid_0's mape: 0.484062
[206]	valid_0'

[453]	valid_0's mape: 0.520126
[454]	valid_0's mape: 0.521062
[455]	valid_0's mape: 0.521494
[456]	valid_0's mape: 0.520874
[457]	valid_0's mape: 0.520677
[458]	valid_0's mape: 0.520558
[459]	valid_0's mape: 0.520301
[460]	valid_0's mape: 0.520041
[461]	valid_0's mape: 0.520147
[462]	valid_0's mape: 0.519888
[463]	valid_0's mape: 0.519732
[464]	valid_0's mape: 0.519508
[465]	valid_0's mape: 0.519306
[466]	valid_0's mape: 0.519554
[467]	valid_0's mape: 0.519587
[468]	valid_0's mape: 0.519777
[469]	valid_0's mape: 0.520049
[470]	valid_0's mape: 0.519959
[471]	valid_0's mape: 0.520327
[472]	valid_0's mape: 0.520297
[473]	valid_0's mape: 0.520696
[474]	valid_0's mape: 0.521261
[475]	valid_0's mape: 0.52112
[476]	valid_0's mape: 0.521401
[477]	valid_0's mape: 0.52159
[478]	valid_0's mape: 0.521211
[479]	valid_0's mape: 0.52139
[480]	valid_0's mape: 0.521492
[481]	valid_0's mape: 0.521228
[482]	valid_0's mape: 0.521529
[483]	valid_0's mape: 0.521116
[484]	valid_0's mape: 0.521207
[485]	valid

In [23]:
scores

[0.59310294503083594]